# Math work

In [1]:
#from PIL import Image
#import sys
#sys.path.append('/usr/local/lib/python3.7/site-packages')
#import os
import numpy as np
import importlib
from numpy import linalg
import PO_helperfuns
from PO_helperfuns import *
import PO_DatFrameFuns
from PO_DatFrameFuns import *
import scipy.stats as scs
import matplotlib as mpl
import matplotlib.pyplot as plt
#importlib.reload(helperfuns)
import pandas as pd
import sympy as sp
from sympy import *
from sympy.abc import lamda, beta
from sympy.solvers import solve
#np.set_printoptions(precision=3, suppress = True)
#import seaborn as sns
# next two libraries are used to flatten list of lists
import functools
import operator
# for formatting tick labels
#from matplotlib.ticker import FormatStrFormatter

#for parallelizing:
import multiprocessing as mp
from pathos.multiprocessing import ProcessingPool as Pool


from sympy.abc import beta, lamda


# Equilibrium

In [2]:
u1,u2,bu,K,pc,beta = sp.symbols('u_1, u_2, \\bar{u}, K, \pi_C, \\beta')

In [83]:
r1 = 1 - beta*u1
r2 = 1 - beta*u2
top = (K*u1*(r1 + r2) + pc*r1*(u1+u2))*(1+r1)
bottom = (K*u2*(r1 + r2) + pc*r2*(u1 + u2))*(1 + r2)
expr = u1*(bottom) - u2*top

In [86]:
factor(expr)

-(u_1 - u_2)*(K*\beta**2*u_1**2*u_2 + K*\beta**2*u_1*u_2**2 - 2*K*\beta*u_1*u_2 + \beta**2*\pi_C*u_1**2*u_2 + \beta**2*\pi_C*u_1*u_2**2 - 2*\pi_C*u_1 - 2*\pi_C*u_2)

In [3]:
u1, u2,bu,r1,r2, K, pc,  W = sp.symbols('\hat{u}_1,\hat{u}_2, \hat{\\bar{u}},\hat{r}_1,\hat{r}_2,  K, \pi_C, \hat{W}')



In [14]:
bu

\hat{\bar{u}}

In [54]:
psi1 = u1/(u1 + u2)
psi2 = u2/(u1+u2)
R1 = r1/(r1+r2)
R2 = r2/(r1+r2)
L1 = K*psi1 + pc*R1
L2 = K*psi2 + pc*R2
What = 1 + (r1)*L1 + (r2)*L2
Wu1 = L1*(2-beta*u1)

In [55]:
expr = What*u1 - Wu1
#expr = expr.subs([(r1,1-beta*u1),(r2,1-beta*u2)])

In [56]:
n,d = fraction(simplify(expr)) # n = 0 is one of the equations we solve

In [57]:
expr

\hat{u}_1*(\hat{r}_1*(K*\hat{u}_1/(\hat{u}_1 + \hat{u}_2) + \hat{r}_1*\pi_C/(\hat{r}_1 + \hat{r}_2)) + \hat{r}_2*(K*\hat{u}_2/(\hat{u}_1 + \hat{u}_2) + \hat{r}_2*\pi_C/(\hat{r}_1 + \hat{r}_2)) + 1) - (-\hat{u}_1*beta + 2)*(K*\hat{u}_1/(\hat{u}_1 + \hat{u}_2) + \hat{r}_1*\pi_C/(\hat{r}_1 + \hat{r}_2))

## Find $J^*$

In [58]:
du1, du2, dr1, dr2 = sp.symbols('\Delta_{u_1}, \Delta_{u_2}, \Delta_{r_1}, \Delta_{r_2}')

In [59]:
dps1 = (du1*u2 - du2*u1)/(u1+u2)**2
dps2 = -dps1
dR1 = (dr1*r2 - dr2*r1)/(r1+r2)**2
dR2 = - dR1
L1 = K*psi1 + pc*R1
dL1 = K*dps1 + pc*dR1
L2 = K*psi2 + pc*R2
dL2 = - dL1

In [71]:
L1, L2 = sp.symbols('L_1, L_2')
dW =r1*dL1  + dr1*L1 + dr2*L2+ r2*dL2

In [78]:
expand(dW).collect([du1,du2,dr1,dr2])

K*\Delta_{u_1}*\hat{r}_1*\hat{u}_2/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2) - K*\Delta_{u_1}*\hat{r}_2*\hat{u}_2/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2) + \Delta_{r_1}*(L_1 + \hat{r}_1*\hat{r}_2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2) - \hat{r}_2**2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2)) + \Delta_{r_2}*(L_2 - \hat{r}_1**2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2) + \hat{r}_1*\hat{r}_2*\pi_C/(\hat{r}_1**2 + 2*\hat{r}_1*\hat{r}_2 + \hat{r}_2**2)) + \Delta_{u_2}*(-K*\hat{r}_1*\hat{u}_1/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2) + K*\hat{r}_2*\hat{u}_1/(\hat{u}_1**2 + 2*\hat{u}_1*\hat{u}_2 + \hat{u}_2**2))

In [67]:
du1 = (1/W)*(L1*dr1 + dL1*(1+r1)) - (dW/(W**2))*(W*u1)

In [69]:
expand(du1).collect([du1,du2,dr1,dr2])

-K*\Delta_{u_1}*\hat{r}_1*\hat{u}_1*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + K*\Delta_{u_1}*\hat{r}_1*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + K*\Delta_{u_1}*\hat{r}_2*\hat{u}_1*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + K*\Delta_{u_1}*\hat{u}_2/(\hat{W}*\hat{u}_1**2 + 2*\hat{W}*\hat{u}_1*\hat{u}_2 + \hat{W}*\hat{u}_2**2) + \Delta_{r_1}*(-K*\hat{u}_1**2/(\hat{W}*\hat{u}_1 + \hat{W}*\hat{u}_2) + K*\hat{u}_1/(\hat{W}*\hat{u}_1 + \hat{W}*\hat{u}_2) - \hat{r}_1*\hat{r}_2*\hat{u}_1*\pi_C/(\hat{W}*\hat{r}_1**2 + 2*\hat{W}*\hat{r}_1*\hat{r}_2 + \hat{W}*\hat{r}_2**2) + \hat{r}_1*\hat{r}_2*\pi_C/(\hat{W}*\hat{r}_1**2 + 2*\hat{W}*\hat{r}_1*\hat{r}_2 + \hat{W}*\hat{r}_2**2) - \hat{r}_1*\hat{u}_1*\pi_C/(\hat{W}*\hat{r}_1 + \hat{W}*\hat{r}_2) + \hat{r}_1*\pi_C/(\hat{W}*\hat{r}_1 + \hat{W}*\hat{r}_2) + \hat{r}_2**2*\hat{u}_1*\pi_C/(\hat{W}*\hat{r}_1**2 + 2*

# Internal Stability

## Equal equilibrium

In [40]:
#L = K + pc; R = 1 + r1
L,R = sp.symbols('L,R')
A = K/2*L
B = (1-u1)*L/(2*W) + pc*(R)/(4*W*r1)
C = -u1*L/(2*W) - pc*(R)/(4*W*r1)
D = -beta*r1/(R)
E = 1/(R)

In [41]:
J = Matrix([[A,-A,B,-B],[-A,A,-B,B],[C,0,E,0],[0,C,0,E]])

In [42]:
charpoly = J.charpoly(lamda)
charpoly = charpoly.as_expr()

In [43]:
charpoly

lamda**4 + lamda**3*(-K*L*R - 2)/R + lamda**2*(16*K*L*R*\hat{W}**2*\hat{r}_1**2 - 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1**2 + 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1 + 2*L*R**3*\hat{r}_1*\pi_C + R**4*\pi_C**2 + 8*\hat{W}**2*\hat{r}_1**2)/(8*R**2*\hat{W}**2*\hat{r}_1**2) + lamda*(-8*K*L*\hat{W}**2*\hat{r}_1**2 + 4*L**2*R*\hat{r}_1**2*\hat{u}_1**2 - 4*L**2*R*\hat{r}_1**2*\hat{u}_1 - 2*L*R**2*\hat{r}_1*\pi_C - R**3*\pi_C**2)/(8*R**2*\hat{W}**2*\hat{r}_1**2)

In [44]:
term2 = charpoly.coeff(lamda**2)
n,d = fraction(term2)

In [48]:
expand(n)

16*K*L*R*\hat{W}**2*\hat{r}_1**2 - 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1**2 + 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1 + 2*L*R**3*\hat{r}_1*\pi_C + R**4*\pi_C**2 + 8*\hat{W}**2*\hat{r}_1**2

In [34]:
term1 = charpoly.coeff(lamda)
n,d = fraction(term1)

In [55]:
evals = J.eigenvals()

In [70]:
listevals = list(evals)
print('first eigenvalue:\n')
display(listevals[0])
print('second eigenvalue:\n')
display(listevals[1])
print('third eigenvalue:\n')
display(listevals[2])
print('fourth eigenvalue:\n')
display(listevals[3])

first eigenvalue:



1/R

second eigenvalue:



(2*K*L*R*\hat{W}*\hat{r}_1 + 2*\hat{W}*\hat{r}_1 + sqrt(2)*sqrt(2*K**2*L**2*R**2*\hat{W}**2*\hat{r}_1**2 - 4*K*L*R*\hat{W}**2*\hat{r}_1**2 + 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1**2 - 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1 - 2*L*R**3*\hat{r}_1*\pi_C - R**4*\pi_C**2 + 2*\hat{W}**2*\hat{r}_1**2))/(4*R*\hat{W}*\hat{r}_1)

third eigenvalue:



(2*K*L*R*\hat{W}*\hat{r}_1 + 2*\hat{W}*\hat{r}_1 - sqrt(2)*sqrt(2*K**2*L**2*R**2*\hat{W}**2*\hat{r}_1**2 - 4*K*L*R*\hat{W}**2*\hat{r}_1**2 + 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1**2 - 4*L**2*R**2*\hat{r}_1**2*\hat{u}_1 - 2*L*R**3*\hat{r}_1*\pi_C - R**4*\pi_C**2 + 2*\hat{W}**2*\hat{r}_1**2))/(4*R*\hat{W}*\hat{r}_1)

fourth eigenvalue:



0

In [100]:
A,B,C,D,E = sp.symbols('A, B, C, D, E')
J = Matrix([[A,-A,B,C],[-A,A,C,B],[D,0,E,0],[0,D,0,E]])
charpoly = J.charpoly(lamda)
charpoly = charpoly.as_expr()

In [8]:
print(factor(charpoly))

-(B*D + C*D + E*lamda - lamda**2)*(2*A*E - 2*A*lamda - B*D + C*D - E*lamda + lamda**2)


In [7]:
evals = list(J.eigenvals())
display(evals[0])
display(evals[1])
display(evals[2])
display(evals[3])

E/2 - sqrt(4*B*D + 4*C*D + E**2)/2

E/2 + sqrt(4*B*D + 4*C*D + E**2)/2

A + E/2 - sqrt(4*A**2 - 4*A*E + 4*B*D - 4*C*D + E**2)/2

A + E/2 + sqrt(4*A**2 - 4*A*E + 4*B*D - 4*C*D + E**2)/2

In [101]:
Q2 = 2*A*E - 2*A*lamda - B*D + C*D - E*lamda + lamda**2

In [102]:
Q2

2*A*E - 2*A*lamda - B*D + C*D - E*lamda + lamda**2

In [103]:
diff_Q2 = diff(Q2,lamda)
lamda_twiddle = solve(diff_Q2,lamda)[0]

In [104]:
Q2_min = Q2.subs([(lamda,lamda_twiddle)])

In [105]:
Q2_min = simplify(Q2_min)

-A**2 + A*E - B*D + C*D - E**2/4

In [108]:
Q2_min_sub = simplify(Q2_min.subs([(A,-K*(1+r1)/(4*W*u1)),(B,pc*(1+r1)/(4*W*r1) - (u1/W)*L),
             (C,-pc*(1+r1)/(4*W*r1) - (u1/W)*L),(D,-beta*r1/(1+r1)),(E,1/(1+r1))]))

In [111]:
Q2_min_sub = simplify(expand(Q2_min_sub))
factor(Q2_min_sub)

-(K**2*r_1**4 + 4*K**2*r_1**3 + 6*K**2*r_1**2 + 4*K**2*r_1 + K**2 + 4*K*W*r_1**2*u_1 + 8*K*W*r_1*u_1 + 4*K*W*u_1 + 4*W**2*u_1**2 - 8*W*\beta*\pi_C*r_1**2*u_1**2 - 16*W*\beta*\pi_C*r_1*u_1**2 - 8*W*\beta*\pi_C*u_1**2)/(16*W**2*u_1**2*(r_1 + 1)**2)

In [113]:
n,d = fraction(Q2_min_sub)
# want to show n < 0 for reality
n_sub = n.subs([(r1,1-beta*u1),(W,1+(1-beta*u1)*(K+pc))])

In [126]:
n_sub = simplify(expand(n_sub))
#n_sub.collect(u1)
# this is less than what we would have if u1^5 = 0
n_sub2 = n_sub.subs([(u1**5,0)])
n_sub2.collect(pc)

-K**2*\beta**4*u_1**4 + 4*K**2*\beta**3*u_1**4 + 8*K**2*\beta**3*u_1**3 - 4*K**2*\beta**2*u_1**4 - 20*K**2*\beta**2*u_1**3 - 24*K**2*\beta**2*u_1**2 + 8*K**2*\beta*u_1**3 + 32*K**2*\beta*u_1**2 + 32*K**2*\beta*u_1 - 4*K**2*u_1**2 - 16*K**2*u_1 - 16*K**2 - 4*K*\beta**2*u_1**3 + 8*K*\beta*u_1**3 + 16*K*\beta*u_1**2 - 8*K*u_1**2 - 16*K*u_1 + \pi_C**2*(40*\beta**3*u_1**4 - 4*\beta**2*u_1**4 - 64*\beta**2*u_1**3 + 8*\beta*u_1**3 + 32*\beta*u_1**2 - 4*u_1**2) + \pi_C*(44*K*\beta**3*u_1**4 - 8*K*\beta**2*u_1**4 - 84*K*\beta**2*u_1**3 + 16*K*\beta*u_1**3 + 64*K*\beta*u_1**2 - 8*K*u_1**2 - 16*K*u_1 + 8*\beta**3*u_1**4 - 32*\beta**2*u_1**3 + 8*\beta*u_1**3 + 32*\beta*u_1**2 - 8*u_1**2) - 4*u_1**2

In [69]:
u1, K, pc,r1 = sp.symbols('u_1, K, \pi_C,r_1')
#r1 = 1 - beta*u1
W = 1 + r1*(K + pc)
L = K/2 + pc/2
B = pc*(1+r1)/(4*W*r1) - (u1/W)*L
C = -pc*(1+r1)/(4*W*r1) - (u1/W)*L
D = -beta*r1/(1+r1)
E = 1/(1+r1)


In [70]:
A = -K*(1+r1)/(4*W*u1)
Q2 = 2*A*E - 2*A*lamda - B*D + C*D - E*lamda + lamda**2
intercept = Q2.subs([(lamda,0)])
simplify(intercept)

(-K + \beta*\pi_C*u_1)/(2*u_1*(r_1*(K + \pi_C) + 1))

In [71]:
A = -K/(4*L)
Q2 = 2*A*E - 2*A*lamda - B*D + C*D - E*lamda + lamda**2


lowerbound = Q2.subs([(lamda,-1)])
n,d = fraction(simplify(expand(lowerbound)))

In [72]:
n.collect(r1)

\pi_C*(K*\beta + \beta*\pi_C + r_1**2*(2*K + 2*\pi_C) + r_1*(K*\beta + 4*K + \beta*\pi_C + 4*\pi_C + 2) + 4)

In [73]:
upperbound = Q2.subs([(lamda,1)])
n,d = fraction(simplify(expand(upperbound)))

In [74]:
n.collect(r1)

K*\beta*\pi_C/2 + \beta*\pi_C**2/2 + r_1**2*(2*K**2 + 3*K*\pi_C + \pi_C**2) + r_1*(K*\beta*\pi_C/2 + 2*K + \beta*\pi_C**2/2 + \pi_C)

In [75]:
diffQ2 = diff(Q2,lamda)
lamda_twiddle = solve(diffQ2,lamda)[0]

In [76]:
lamda_twiddle
n,d = fraction(lamda_twiddle)

In [89]:
lamda_twiddle

(-K*r_1/2 + \pi_C/2)/(K*r_1 + K + \pi_C*r_1 + \pi_C)